# Numba

In addition to what's in Anaconda, this lecture will need the following libraries:

In [ ]:
!conda install -y quantecon

In [ ]:
%matplotlib inline
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt

## Overview

In an earlier lecture we learned about vectorization, which is one method to improve speed and efficiency in numerical work.

Vectorization involves sending array processing operations in batch to efficient low-level code.

However, vectorization has several weaknesses.

One is that it is highly memory-intensive when working with large amounts of data.

Another is that the set of algorithms that can be entirely vectorized is not universal.

In fact, for some algorithms, vectorization is ineffective.

Fortunately, a Python library called [Numba](http://numba.pydata.org/) solves many of these problems.

It does so through something called **just in time (JIT) compilation**.

The key idea is to compile functions to native machine code instructions on the fly.

When it succeeds, the compiled code is extremely fast.

This lecture introduces the main ideas.


## Applications

As stated above, Numba's primary use is compiling functions to fast native
machine code during runtime.

### An Example

Let's consider a problem that is difficult to vectorize: generating the trajectory of a difference equation given an initial condition.

We will take the difference equation to be the quadratic map

$$
x_{t+1} = \alpha x_t (1 - x_t)
$$

In what follows we set

In [ ]:
α = 4.0

Here's the plot of a typical trajectory, starting from $x_0 = 0.1$, with $t$ on the x-axis

In [ ]:
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
      x[t+1] = α * x[t] * (1 - x[t])
    return x

x = qm(0.1, 250)
fig, ax = plt.subplots()
ax.plot(x, 'b-', lw=2, alpha=0.8)
ax.set_xlabel('$t$', fontsize=12)
ax.set_ylabel('$x_{t}$', fontsize = 12)
plt.show()

To speed the function `qm` up using Numba, our first step is

In [ ]:
from numba import jit

qm_numba = jit(qm)

The function `qm_numba` is a version of `qm` that is "targeted" for
JIT-compilation.

Let's time and compare identical function calls across these two versions, starting with the original function `qm`:

In [ ]:
n = 10_000_000

qe.tic()
qm(0.1, int(n))
time1 = qe.toc()

Now let's try qm_numba

In [ ]:
qe.tic()
qm_numba(0.1, int(n))
time2 = qe.toc()

This is already a massive speed gain.

In fact, the next time and all subsequent times it runs even faster as the function has been compiled and is in memory:

(qm_numba_result)=

In [ ]:
qe.tic()
qm_numba(0.1, int(n))
time3 = qe.toc()

In [ ]:
time1 / time3  # Calculate speed gain

This kind of speed gain is huge relative to how simple and clear the implementation is.


### How and When it Works

Numba attempts to generate fast machine code using the infrastructure provided by the [LLVM Project](http://llvm.org/).

It does this by inferring type information on the fly.

The basic idea is this:

* Python is very flexible and hence we could call the function qm with many
  types.
    * e.g., `x0` could be a NumPy array or a list, `n` could be an integer or a float, etc.
* This makes it hard to *pre*-compile the function.
* However, when we do actually call the function, say by executing `qm(0.5, 10)`,
  the types of `x0` and `n` become clear.
* Moreover, the types of other variables in `qm` can be inferred once the input is known.
* So the strategy of Numba and other JIT compilers is to wait until this
  moment, and *then* compile the function.

That's why it is called "just-in-time" compilation.

Note that, if you make the call `qm(0.5, 10)` and then follow it with `qm(0.9, 20)`, compilation only takes place on the first call.

The compiled code is then cached and recycled as required.

## Decorators and "nopython" Mode

In the code above we created a JIT compiled version of `qm` via the call

In [ ]:
qm_numba = jit(qm)

In practice this would typically be done using an alternative *decorator* syntax.

(We will explain all about decorators in a {doc}`later lecture <python_advanced_features>` but you can skip the details at this stage.)

Let's see how this is done.

### Decorator Notation

To target a function for JIT compilation we can put `@jit` before the function definition.

Here's what this looks like for `qm`

In [ ]:
@jit
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = α * x[t] * (1 - x[t])
    return x

This is equivalent to `qm = jit(qm)`.

The following now uses the jitted version:

In [ ]:
qm(0.1, 10)

### Type Inference and "nopython" Mode

Clearly type inference is a key part of JIT compilation.

As you can imagine, inferring types is easier for simple Python objects (e.g., simple scalar data types such as floats and integers).

Numba also plays well with NumPy arrays.

In an ideal setting, Numba can infer all necessary type information.

This allows it to generate native machine code, without having to call the Python runtime environment.

In such a setting, Numba will be on par with machine code from low-level languages.

When Numba cannot infer all type information, some Python objects are given generic object status and execution falls back to the Python runtime.

When this happens, Numba provides only minor speed gains or none at all.

We generally prefer to force an error when this occurs, so we know effective
compilation is failing.

This is done by using either `@jit(nopython=True)` or, equivalently, `@njit` instead of `@jit`.

For example,

In [ ]:
from numba import njit

@njit
def qm(x0, n):
    x = np.empty(n+1)
    x[0] = x0
    for t in range(n):
        x[t+1] = 4 * x[t] * (1 - x[t])
    return x

## Ex. 1

Previously we considered how to approximate $\pi$ by
Monte Carlo.

```{code-cell python3}
from random import uniform

def calculate_pi(n=1_000_000):
    count = 0
    for i in range(n):
        u, v = uniform(0, 1), uniform(0, 1)
        d = np.sqrt((u - 0.5)**2 + (v - 0.5)**2)
        if d < 0.5:
            count += 1

    area_estimate = count / n
    return area_estimate * 4  # dividing by radius**2
```

Make the code efficient using Numba.

Compare speed with and without Numba when the sample size is large.